**Sentiment Analysis**

**I. Individual Analysis**

**Decision Trees**

*General Imports*

In [1]:
import nltk
from random import shuffle
from sklearn.linear_model import LogisticRegression
from nltk.metrics.scores import (precision, recall, accuracy)
from sklearn.tree import DecisionTreeClassifier
#from sklearn.ensemble import RandomForestClassifier

**Features Construction**

Useful functions to load the different features models

In [2]:
def load_bow(thefeatures,filename):
    '''
    Loads the tags in the filename and appends them to the
    features list.
    '''
    f = open(filename, "r")
    line = f.readline()
    while len(line) >0:
        splited = line.split(' ')
        feats = {}
        for word in range(len(splited)-1):
            theword = splited[word].split(":")
            feats[theword[0]] = int(theword[1])
        tag = splited[-1].split("#:")[1][0:3]
        thefeatures.append((feats,tag))
        line = f.readline()
    f.close()

In [3]:
def load_binary_bow(thefeatures,filename):
    '''
    Loads the tags in the filename and appends them to the
    features list. Filters them to change it to 1 for each 
    existent element in the Bag of Words.
    '''
    f = open(filename, "r")
    line = f.readline()
    while len(line) >0:
        splited = line.split(' ')
        feats = {}
        for word in range(len(splited)-1):
            theword = splited[word].split(":")
            feats[theword[0]] = 1
        tag = splited[-1].split("#:")[1][0:3]
        thefeatures.append((feats,tag))
        line = f.readline()
    f.close()

In [4]:
def load_lexicon(thefeatures,filename):
    '''
    Loads the tags process them to create the Features based on the
    AFINN-111 sentiment lexicon.
    '''
    lex = load_affin()
    f = open(filename, "r")
    line = f.readline()
    while len(line) >0:
        splited = line.split(' ')
        feats1 = {}
        summa = 0
        negga = 0
        pos = 0
        neg = 0
        tot = 0
        
        for word in range(len(splited)-1):
            add = False
            theword = splited[word].split(":")
            lilwords = theword[0].split("_")
            for lw in lilwords:
                if lw in lex:
                    add = True
                    if lex[lw] > 0:
                        summa += lex[lw]*int(theword[1])
                        pos += int(theword[1])
                    else:
                        negga += lex[lw]*int(theword[1])
                        neg += int(theword[1])
            tot += int(theword[1])
        feats1["point_pos"] = summa
        feats1["point_neg"] = -negga
        feats1["count_pos"] = pos
        feats1["count_neg"] = neg
        feats1["size"] = tot
        feats1["porc_pos"] = pos/tot
        feats1["porc_neg"] = neg/tot
        tag = splited[-1].split("#:")[1][0:3]
        thefeatures.append((feats1,tag))
        line = f.readline()
    f.close()

def load_affin():
    '''
    Loads the lexicon of sentiment words included in the AFINN-11 lexicon
    '''
    f = open("AFINN-111.txt", "r")
    line = f.readline()
    affin = {}
    while len(line) >0:
        splited = line.split('\t')
        affin[splited[0]] = int(splited[1])
        line = f.readline()
    return affin

*Additional Function*

Shows the most informative features for the Linear Regression Models.

Taken from the answers to: https://stackoverflow.com/questions/11116697/how-to-get-most-informative-features-for-scikit-learn-classifiers Kudos to him.

In [5]:
def show_most_informative_features(vectorizer, clf, n=20):
    '''
    Gets the most informative features from a Linear Regression Model
    '''
    feature_names = vectorizer.get_feature_names()
    coefs_with_fns = sorted(zip(clf.feature_importances_, feature_names))
    top = zip(coefs_with_fns[:n], coefs_with_fns[:-(n + 1):-1])
    for (coef_1, fn_1), (coef_2, fn_2) in top:
        print ("\t%.4f\t%-15s\t\t%.4f\t%-15s" % (coef_1, fn_1, coef_2, fn_2))

**BoW**

Loads the data and creates the feature to train. Loads and uses both review files.

In [6]:
BoWFeatures = []

#Loads the positive reviews
load_bow(BoWFeatures,"reviews/books/positive.review")
load_bow(BoWFeatures,"reviews/dvd/positive.review")
load_bow(BoWFeatures,"reviews/electronics/positive.review")
load_bow(BoWFeatures,"reviews/kitchen/positive.review")

#Loads the negative reviews 
load_bow(BoWFeatures,"reviews/books/negative.review")
load_bow(BoWFeatures,"reviews/dvd/negative.review")
load_bow(BoWFeatures,"reviews/electronics/negative.review")
load_bow(BoWFeatures,"reviews/kitchen/negative.review")

print("Loaded",len(BoWFeatures),"reviews")

Loaded 8000 reviews


Loads the data and creates the feature to test the classifier. 

In [7]:
BoWTesting =[]

#Loads the testing reviews 
load_bow(BoWTesting,"reviews/books/unlabeled.review")
load_bow(BoWTesting,"reviews/dvd/unlabeled.review")
load_bow(BoWTesting,"reviews/electronics/unlabeled.review")
load_bow(BoWTesting,"reviews/kitchen/unlabeled.review")

print("Loaded",len(BoWTesting),"reviews")

Loaded 19677 reviews


**Binary BoW**

Loads the data and creates the feature to train. Loads and uses both review files. Filters them to change it to 1 for each existent element in the Bag of Words.

In [8]:
BinaryBoWFeatures = []

#Loads the positive reviews
load_binary_bow(BinaryBoWFeatures,"reviews/books/positive.review")
load_binary_bow(BinaryBoWFeatures,"reviews/dvd/positive.review")
load_binary_bow(BinaryBoWFeatures,"reviews/electronics/positive.review")
load_binary_bow(BinaryBoWFeatures,"reviews/kitchen/positive.review")

#Loads the negative reviews 
load_binary_bow(BinaryBoWFeatures,"reviews/books/negative.review")
load_binary_bow(BinaryBoWFeatures,"reviews/dvd/negative.review")
load_binary_bow(BinaryBoWFeatures,"reviews/electronics/negative.review")
load_binary_bow(BinaryBoWFeatures,"reviews/kitchen/negative.review")

print("Loaded",len(BinaryBoWFeatures),"reviews")

Loaded 8000 reviews


Loads the data and creates the feature to test the classifier. 

In [9]:
BinaryBoWTesting =[]

#Loads the testing reviews 
load_binary_bow(BinaryBoWTesting,"reviews/books/unlabeled.review")
load_binary_bow(BinaryBoWTesting,"reviews/dvd/unlabeled.review")
load_binary_bow(BinaryBoWTesting,"reviews/electronics/unlabeled.review")
load_binary_bow(BinaryBoWTesting,"reviews/kitchen/unlabeled.review")

print("Loaded",len(BinaryBoWTesting),"reviews")

Loaded 19677 reviews


**Lexicon Features**

Loads the data and creates the feature to train. Loads and uses both review files. Takes into use the following Features:

- Number of positive words
- Number of negative words
- Positive score following the AFINN-111 scores
- Negative score following the AFINN-111 scores
- Size (number of words) of the review
- Percentage of positive words
- Percentage of negative words

In [10]:
LexiconFeatures = []

#Loads the positive reviews
load_lexicon(LexiconFeatures,"reviews/books/positive.review")
load_lexicon(LexiconFeatures,"reviews/dvd/positive.review")
load_lexicon(LexiconFeatures,"reviews/electronics/positive.review")
load_lexicon(LexiconFeatures,"reviews/kitchen/positive.review")

#Loads the negative reviews 
load_lexicon(LexiconFeatures,"reviews/books/negative.review")
load_lexicon(LexiconFeatures,"reviews/dvd/negative.review")
load_lexicon(LexiconFeatures,"reviews/electronics/negative.review")
load_lexicon(LexiconFeatures,"reviews/kitchen/negative.review")

print("Loaded",len(LexiconFeatures),"reviews")

Loaded 8000 reviews


Loads the data and creates the feature to test the classifier. 

In [11]:
LexiconTesting =[]

#Loads the testing reviews 
load_lexicon(LexiconTesting,"reviews/books/unlabeled.review")
load_lexicon(LexiconTesting,"reviews/dvd/unlabeled.review")
load_lexicon(LexiconTesting,"reviews/electronics/unlabeled.review")
load_lexicon(LexiconTesting,"reviews/kitchen/unlabeled.review")

print("Loaded",len(LexiconTesting),"reviews")

Loaded 19677 reviews


**Testing Zone**

Now we are gonna test the three different Features representations with the decision tree.

**BoW - Decision Tree**

Trains the dataset over a Decision Tree with a Bag of Words Model.

**Training**

In [12]:
shuffle(BoWFeatures)
classifier = nltk.classify.SklearnClassifier(DecisionTreeClassifier())
classifier.train(BoWFeatures)

<SklearnClassifier(DecisionTreeClassifier())>

**Validating**

In [13]:
refsets = nltk.collections.defaultdict(set)
testsets = nltk.collections.defaultdict(set)
real = []
result = []

for i, (feats, label) in enumerate(BoWFeatures):
    refsets[label].add(i)
    observed = classifier.classify(feats)
    testsets[observed].add(i)
    real.append(label)
    result.append(observed)

print( 'For Validating set:')
print( 'Accuracy:', nltk.accuracy(real,result) )
print( 'Precision:', nltk.scores.precision(refsets['pos'], testsets['pos']) )
print( 'Recall:', nltk.recall(refsets['pos'], testsets['pos']) )
print( 'F1 Score:', nltk.f_measure(refsets['pos'], testsets['pos']) )

For Validating set:
Accuracy: 1.0
Precision: 1.0
Recall: 1.0
F1 Score: 1.0


**Testing**

In [14]:
refsets = nltk.collections.defaultdict(set)
testsets = nltk.collections.defaultdict(set)
real = []
result = []

for i, (feats, label) in enumerate(BoWTesting):
    refsets[label].add(i)
    observed = classifier.classify(feats)
    testsets[observed].add(i)
    real.append(label)
    result.append(observed)

print( 'For testing set:')
print( 'Accuracy:', nltk.accuracy(real,result) )
print( 'Precision:', nltk.scores.precision(refsets['pos'], testsets['pos']) )
print( 'Recall:', nltk.recall(refsets['pos'], testsets['pos']) )
print( 'F1 Score:', nltk.f_measure(refsets['pos'], testsets['pos']) )

For testing set:
Accuracy: 0.7240432992834274
Precision: 0.7201780415430267
Recall: 0.7367941712204007
F1 Score: 0.728391356542617


In [15]:
show_most_informative_features(classifier._vectorizer,classifier._clf,10)  

	0.0000	?maybe        		0.0533	great          
	0.0000	?maybe_i      		0.0388	not            
	0.0000	!!!!!--over    		0.0347	excellent      
	0.0000	!!!!!--over_rated		0.0252	waste          
	0.0000	!!!!finally    		0.0208	best           
	0.0000	!!!!finally_i  		0.0177	love           
	0.0000	!ps            		0.0163	easy           
	0.0000	!ps_use        		0.0154	bad            
	0.0000	"              		0.0125	worst          
	0.0000	"$9            		0.0078	a_must         


**Binary BoW - Decision Tree**

Trains the dataset over a Decision Tree with a Binary Bag of Words Model.

**Training**

In [16]:
shuffle(BinaryBoWFeatures)
classifier = nltk.classify.SklearnClassifier(DecisionTreeClassifier())
classifier.train(BinaryBoWFeatures)

<SklearnClassifier(DecisionTreeClassifier())>

**Validating**

In [17]:
refsets = nltk.collections.defaultdict(set)
testsets = nltk.collections.defaultdict(set)
real = []
result = []

for i, (feats, label) in enumerate(BinaryBoWFeatures):
    refsets[label].add(i)
    observed = classifier.classify(feats)
    testsets[observed].add(i)
    real.append(label)
    result.append(observed)

print( 'For Validating set:')
print( 'Accuracy:', nltk.accuracy(real,result) )
print( 'Precision:', nltk.scores.precision(refsets['pos'], testsets['pos']) )
print( 'Recall:', nltk.recall(refsets['pos'], testsets['pos']) )
print( 'F1 Score:', nltk.f_measure(refsets['pos'], testsets['pos']) )

For Validating set:
Accuracy: 1.0
Precision: 1.0
Recall: 1.0
F1 Score: 1.0


**Testing**

In [18]:
refsets = nltk.collections.defaultdict(set)
testsets = nltk.collections.defaultdict(set)
real = []
result = []

for i, (feats, label) in enumerate(BinaryBoWTesting):
    refsets[label].add(i)
    observed = classifier.classify(feats)
    testsets[observed].add(i)
    real.append(label)
    result.append(observed)

print( 'For testing set:')
print( 'Accuracy:', nltk.accuracy(real,result) )
print( 'Precision:', nltk.scores.precision(refsets['pos'], testsets['pos']) )
print( 'Recall:', nltk.recall(refsets['pos'], testsets['pos']) )
print( 'F1 Score:', nltk.f_measure(refsets['pos'], testsets['pos']) )

For testing set:
Accuracy: 0.7286171672511054
Precision: 0.7287009063444109
Recall: 0.73224043715847
F1 Score: 0.7304663840096911


In [19]:
show_most_informative_features(classifier._vectorizer,classifier._clf,10)  

	0.0000	?maybe        		0.0533	great          
	0.0000	?maybe_i      		0.0353	excellent      
	0.0000	!!!!!--over    		0.0339	not            
	0.0000	!!!!!--over_rated		0.0247	waste          
	0.0000	!!!!finally    		0.0208	best           
	0.0000	!!!!finally_i  		0.0177	love           
	0.0000	!ps            		0.0173	bad            
	0.0000	!ps_use        		0.0167	easy           
	0.0000	"              		0.0125	worst          
	0.0000	"$9            		0.0078	a_must         


**Lexicon Features - Decision Tree**

Trains the dataset over a Decision Tree with a Lexicon Features Model.

**Training**

In [20]:
shuffle(LexiconFeatures)
classifier = nltk.classify.SklearnClassifier(DecisionTreeClassifier())
classifier.train(LexiconFeatures)

<SklearnClassifier(DecisionTreeClassifier())>

**Validating**

In [21]:
refsets = nltk.collections.defaultdict(set)
testsets = nltk.collections.defaultdict(set)
real = []
result = []

for i, (feats, label) in enumerate(LexiconFeatures):
    refsets[label].add(i)
    observed = classifier.classify(feats)
    testsets[observed].add(i)
    real.append(label)
    result.append(observed)

print( 'For Validating set:')
print( 'Accuracy:', nltk.accuracy(real,result) )
print( 'Precision:', nltk.scores.precision(refsets['pos'], testsets['pos']) )
print( 'Recall:', nltk.recall(refsets['pos'], testsets['pos']) )
print( 'F1 Score:', nltk.f_measure(refsets['pos'], testsets['pos']) )

For Validating set:
Accuracy: 0.98875
Precision: 0.9972024415055951
Recall: 0.98025
F1 Score: 0.9886535552193646


**Testing**

In [22]:
refsets = nltk.collections.defaultdict(set)
testsets = nltk.collections.defaultdict(set)
real = []
result = []

for i, (feats, label) in enumerate(LexiconTesting):
    refsets[label].add(i)
    observed = classifier.classify(feats)
    testsets[observed].add(i)
    real.append(label)
    result.append(observed)

print( 'For testing set:')
print( 'Accuracy:', nltk.accuracy(real,result) )
print( 'Precision:', nltk.scores.precision(refsets['pos'], testsets['pos']) )
print( 'Recall:', nltk.recall(refsets['pos'], testsets['pos']) )
print( 'F1 Score:', nltk.f_measure(refsets['pos'], testsets['pos']) )

For testing set:
Accuracy: 0.6583320628144534
Precision: 0.6625501698054955
Recall: 0.6514875531268974
F1 Score: 0.6569722945048218


In [23]:
show_most_informative_features(classifier._vectorizer,classifier._clf,10)  

	0.0378	count_neg      		0.2876	porc_neg       
	0.0634	count_pos      		0.2520	porc_pos       
	0.0865	point_neg      		0.1364	point_pos      
	0.1363	size           		0.1363	size           
	0.1364	point_pos      		0.0865	point_neg      
	0.2520	porc_pos       		0.0634	count_pos      
	0.2876	porc_neg       		0.0378	count_neg      


**Table of Results - Unified Data**

| Combination | Accuracy | Precision | Recall | F1 Score|
| --- | ----------- |----------- |----------- |----------- |
| **BOW - DT** | 0.7240432992834274 | 0.7201780415430267 | **0.7367941712204007** | 0.728391356542617 |
| **BBOW - DT** | **0.7286171672511054** | **0.7287009063444109** | 0.73224043715847 | **0.7304663840096911** |
| **Lexicon - DT** | 0.6583320628144534 | 0.6625501698054955 | 0.6514875531268974 | 0.6569722945048218 |

